# Practical Assignment: Neighbourhood Cyclability Analysis

In [ ]:
#import libraries

import statsmodels.formula.api as smf
import csv
import psycopg2
import psycopg2.extras
import shapefile
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import requests
from bs4 import BeautifulSoup
from geopandas import GeoSeries, GeoDataFrame
from shapely.wkt import loads
import descartes
from shapely.geometry import Point, Polygon
import shapely
import geopandas as gpd
import ast
import seaborn as sns

In [ ]:
#Function for server connection. Used from DATA2901 week 9 lab jupyter notebook

def pgconnect(unikey,pw):
    YOUR_UNIKEY = unikey
    YOUR_PW     = pw
    try: 
        conn = psycopg2.connect(host='soit-db-pro-1.ucc.usyd.edu.au',
                                database='y19s1d2x01_'+YOUR_UNIKEY,
                                user='y19s1d2x01_'+YOUR_UNIKEY, 
                                password=YOUR_PW)
        
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
    return conn

In [ ]:
# Function used from DATA2901 week 9 lab jupyter notebook. Excecute commits and queries. 
#Connect to the server and to Public Schema.

#

conn = pgconnect("mile3901", "480133780")



#function to commit our SWL statements
def pgexec( conn, sqlcmd, args, msg, silent=False ):
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
               cur.execute(sqlcmd)
            else:
               cur.execute(sqlcmd, args)
            if silent == False: 
                print("success: " + msg)
            retval = True
         except Exception as e:
            if silent == False: 
                print("db error: ")
                print(e)
   return retval

def pgquery( conn, sqlcmd, args=None, msg=False, returntype='tuple'):
    """ utility function to execute some SQL query statement
        it can take optional arguments (as a dictionary) to fill in for placeholders in the SQL
        will return the complete query result as return value - or in case of error: None
        error and transaction handling built-in (by using the 'with' clauses)"""
    retval = None
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if msg != False:
                    print("success: " + msg)
            except psycopg2.DatabaseError as e:
                if e.pgcode != None and msg:
                    print("db read error: "+msg)
                    print(e)
            except Exception as e:
                print(e)
    return retval
pgexec(conn, "set search_path to public", None, "set the working space to public")

# Task 1: Data Integration and Database Generation

In [ ]:
#Importing and Cleaning Datasets

pd = open("Neighbourhoods.csv",'r')
file = open("Clean_Neighbourhoods.csv", 'w')
flag = False#first row

for i in pd:
    i = i.split(",")


    if len(i) == 6 and flag:#if no missing value and not first row

        if i[5] == "\n":
            i[5] = "0\n"#last value is empty then just replace it with 0
        elif i[5][0] == "-": #if the value is less than 0 then it's a incorrect value
            i[5] = "-1\n"

        for j in range(2,len(i)-1):
            if i[j] == "":#if no value
                i[j] = "0"
            elif i[j][0] == "-":#if incorrect value
                i[j] = "-1"

        if i[1] == "":
            i[1] = "NULL"

    file.write("{},{},{},{},{},{}".format(i[0],i[1],i[2],i[3],i[4],i[5]))
    flag = True#This was the first row
    


In [ ]:
#Importing and Cleaning Datasets

pd = open("BusinessStats.csv",'r')
file = open("Clean_BusinessStats.csv", 'w+')
flag = False#first row

for i in pd:
    i = i.split(",")


    if len(i) == 7 and flag:#if no missing value and not first row

        if i[6] == "\n":
            i[6] = "0\n"
        elif i[6][0] == "-":
            i[6] = "-1\n"

        for j in range(1,len(i)-1):
            if i[j] == "":#if no value
                i[j] = "0"
            elif i[j][0] == "-":#if incorrect value
                i[j] = "-1"



    file.write("{},{},{},{},{},{},{}".format(i[0],i[1],i[2],i[3],i[4],i[5],i[6]))
    flag = True#This was the first row


In [ ]:
#Importing and Cleaning Datasets

pd = open("CensusStats.csv",'r')
file = open("Clean_CensusStats.csv", 'w+')
flag = False

for i in pd:

    i = i.split(',')

    if len(i) == 3 and flag:

        if i[1] == '':
            i[1] = '0'
        elif i[1][0] == '-':
            i[i] ='0';


        if i[2] == "\n":
            i[2] = "0\n"
        elif i[2][0] == "-":
            i[2] = "-1\n"


    flag = True
    file.write("{},{},{}".format(i[0],i[1],i[2]))   

In [ ]:
#Storing data in dictionaries

data_BikeSharingPods = list(csv.DictReader(open("BikeSharingPods.csv")))
data_BusinessStats = list(csv.DictReader(open("Clean_BusinessStats.csv")))
data_Neighbourhood = list(csv.DictReader(open("Clean_Neighbourhoods.csv")))
data_CensusStats = list(csv.DictReader(open("Clean_CensusStats.csv")))
data_StatisticalAreas = list(csv.DictReader(open("StatisticalAreas.csv")))
print(data_CensusStats)

In [ ]:
# Drop tables if they exist. Create tables



pgexec(conn, """
drop table if exists StatisticalAreas;
drop table if exists Neighbourhoods;
drop table if exists CensusStats;
drop table if exists BusinessStats;
drop table if exists BikeSharingPods;
""", None, "Drop tables if they exists")

pgexec(conn, """create table StatisticalAreas(
area_id int primary key,
name varchar(50),
parent_id int
);
create table Neighbourhoods(
area_id int,
name varchar(50) not NULL,
land_area float,
population int,
dwellings int,
business int,
foreign key (area_id) references StatisticalAreas(area_id)
);

create table CensusStats(
area_id int,
income int,
rent int,
foreign key (area_id) references StatisticalAreas(area_id)
);

create table BusinessStats(
area_id int,
num_business int,
retail_trade int,
acc_food int, 
health_social int,
ed_training int,
art int,
foreign key (area_id) references StatisticalAreas(area_id)
);

create table BikeSharingPods(
station_id int primary key,
name varchar(1000),
bike int,
scooter int,
latitude float,
longitude float,
description varchar(1000)

);""", None, "Create tables")



In [ ]:
#Insert data into the table

pgexec(conn, """Truncate table Neighbourhoods;
Truncate table Censusstats;
truncate table businessStats;
truncate table BikeSharingpods;""",None, "Truncate Tables")

for row in data_StatisticalAreas:
    pgexec(conn,"""insert into statisticalareas(area_id,name,parent_id) values(%(area_id)s,%(area_name)s,%(parent_area_id)s)""",row,"Row copied")



In [ ]:
for row in data_BusinessStats:
    pgexec(conn,"""insert into BusinessStats(area_id,num_business, retail_trade, acc_food,health_social,ed_training,art) values(%(area_id)s,%(num_businesses)s,%(retail_trade)s,%(accommodation_and_food_services)s,%(health_care_and_social_assistance)s,%(education_and_training)s,%(arts_and_recreation_services)s)""",row,"Row copied")
    

In [ ]:
for row in data_BikeSharingPods:
    pgexec(conn, "Insert into BikeSharingPods(station_id,name,bike,scooter,latitude,longitude, description) values(%(station_id)s, %(name)s, %(num_bikes)s,%(num_scooters)s,%(latitude)s,%(longitude)s,%(description)s)",row,"Row copied")


In [ ]:
for row in data_Neighbourhood:
    pgexec(conn,"""insert into neighbourhoods(area_id, name,land_area,population,dwellings,business) values(%(area_id)s,%(area_name)s,%(land_area)s,%(population)s,%(number_of_dwellings)s,%(number_of_businesses)s)""",row,"Row copied")

In [ ]:
for row in data_CensusStats:

    pgexec(conn,"""insert into censusstats(area_id,income,rent) values(%(area_id)s,%(median_annual_household_income)s,%(avg_monthly_rent)s)""",row,"Row copied")



In [ ]:
#Uploading Geom Data into a new table called Area

sf = shapefile.Reader("file/SA2_2016_AUST.shp", encoding="iso-8859-1")


if (sf.shapeType == shapefile.POLYGON):
    shapes = sf.shapes()
    print("#shapes: ",len(shapes))
    for shape in shapes:
       print(shape.__geo_interface__['type'], len(shape.points), "points; bounding box:", shape.bbox)
    



In [ ]:
area_schema = '''CREATE TABLE Area (
                     area_id NUMERIC, 
                     fivedig_id NUMERIC,
                     name VARCHAR(100),
                     geom GEOMETRY(Polygon,4283))''' 

pgquery(conn, "DROP TABLE Area", msg="cleared old table")
pgquery(conn, area_schema, msg="created Area table")

In [ ]:
query = '''SELECT area_id FROM Neighbourhoods''' 

list1=pgquery(conn, query, msg="Neighbourhoods")

neighbourhood_list =[]
for i in list1:
    neighbourhood_list.append(i[0])

In [ ]:
import re

insert_stmt = """INSERT INTO Area VALUES ( %(area_id)s, %(fivedig_id)s, %(name)s,
                                            ST_GEOMFROMTEXT(%(geom)s, 4283) )"""

shapes = sf.shapes()
records= sf.records()

row = {}
for i in range(0, len(shapes)):
    record = sf.record(i)
    shape  = sf.shape(i)
    if int(record[0]) in neighbourhood_list:
        row['area_id']=record[0]
        row['fivedig_id']=record[1]
        row['name']=record[2]
        #print(row)

        # prepare the polygon data
        # this is a bit complex with our dataset as it has complex polygons, some with multiple parts...
        row['geom']="POLYGON(("
        i=0
        for x, y in shape.points:
           row['geom']+="%s %s," % (x,y)
           # check for start of a new polygon part
           i += 1
           if i in shape.parts:
               row['geom']= re.sub(",$", "),(", row['geom'])
        # properly end the polygon string
        row['geom'] = re.sub(",$", "))", row['geom'])

        # finally: insert new row into the table
        pgquery(conn, insert_stmt, args=row, msg="inserted "+str(record[2]))

In [ ]:
query='''alter table neighbourhoods
        add geom GEOMETRY(Polygon,4283)
'''

pgquery(conn,query,msg="alter table Neighbourhoods, adding geom column")

query='''    
update neighbourhoods
set geom = area.geom
from area where neighbourhoods.area_id=area.area_id
'''

pgquery(conn,query,msg="update geom column")

In [ ]:
#Alter table Bikesharingpods

query = '''ALTER TABLE BikeSharingPods
               ADD geom_pts GEOMETRY(Point,4283);
               
            '''
pgquery(conn, query, msg="row added")

query = '''UPDATE BikeSharingPods
           SET geom_pts = ST_SetSRID(ST_MakePoint(longitude,latitude),4283)
'''
pgquery(conn, query, msg="row updated")

In [ ]:
#Create Index

query='''drop index if exists Area_id_index;
create unique index Area_id_index on Neighbourhoods(area_id);
drop index if exists Bike_geom_index;
create index Bike_geom_index on Bikesharingpods using  GIST (geom_pts);

'''

pgquery(conn, query, msg="create index")

In [ ]:
#Create table of spatial join

query = '''
            create table spatial_join as
            select area_id,count(station_id) as "num_station", count(station_id)/land_area as "ratio" from Neighbourhoods n
            left outer join Bikesharingpods b ON ST_DWITHIN(N.geom,b.geom_pts,0.02)
            group by area_id, land_area
'''
pgquery(conn, query, msg="spatial join complete")



# Task 2: Cyclability Analysis (Note: Indexes are already created above)

In [ ]:
#drop views if they exists

query ='''
drop view if exists score_final;
drop view if exists score_bike;
drop view if exists score_pop;
drop view if exists score_dwelling;
drop view if exists score_service;


'''
pgquery(conn, query, msg="dropped views")


In [ ]:
#Create views of z-score for each category

#population density
query = '''create view score_pop
as select area_id, name,((n.population/n.land_area) - n1.average)/n1.std as "z_score" from
(select avg(population/land_area) as "average", stddev(population/land_area) as "std" from neighbourhoods)
as n1,
Neighbourhoods n;
'''
pgquery(conn, query, msg="score_pop view")

#Dwelling densinty

query = '''create view score_dwelling
as select n.area_id, name, ((n.dwellings/n.land_area)-n1.average)/ n1.std as "z_score" from
(select avg(dwellings/land_area) as "average", stddev(dwellings/land_area) as "std" from neighbourhoods) as n1,
neighbourhoods n;
'''
pgquery(conn, query, msg="score_dwelling")

#Business service score

query='''
    create view score_service
as select
n.area_id, n.name, ((b.retail_trade/nullif(cast(b.num_business as float),0))-b1.avg_retail)/b1.std_retail as "retail_score",
((b.acc_food/nullif(cast(b.num_business as float),0))-b1.avg_food)/b1.std_food as "food_score",
((b.health_social/nullif(cast(b.num_business as float),0))-b1.avg_health)/b1.std_health as "health_score",
((b.ed_training/nullif(cast(b.num_business as float),0))-b1.avg_ed)/b1.std_ed as "ed_score",
((b.art/nullif(cast(b.num_business as float),0))-b1.avg_art)/b1.std_art as "art_score" from
(select avg(cast(b.retail_trade as float)/nullif(cast(b.num_business as float),0)) as "avg_retail", 
	avg(cast(b.acc_food as float)/nullif(cast(b.num_business as float),0)) as "avg_food",
	avg(cast(b.health_social as float)/nullif(cast(b.num_business as float),0)) as "avg_health",
	avg(cast(b.ed_training as float)/nullif(cast(b.num_business as float),0)) as "avg_ed", 
	avg(cast(b.art as float)/nullif(cast(b.num_business as float),0)) as "avg_art",
	stddev(cast(b.retail_trade as float)/nullif(cast(b.num_business as float),0)) as "std_retail",
	stddev(cast(b.acc_food as float)/nullif(cast(b.num_business as float),0)) as "std_food", 
	stddev(cast(b.health_social as float)/nullif(cast(b.num_business as float),0)) as "std_health",
	stddev(cast(b.ed_training as float)/nullif(cast(b.num_business as float),0)) as "std_ed", 
	stddev(cast(b.art as float)/nullif(cast(b.num_business as float),0)) as "std_art" from businessstats b) as b1, Neighbourhoods n
left outer join businessstats b on n.area_id=b.area_id
'''

pgquery(conn, query, msg="score_service")

#Bikepod z-score

query='''
create view score_bike as
select n.area_id, (b.ratio-b1.average)/b1.std as "score" from 
(select stddev(ratio) as "std", avg(ratio) as "average" from spatial_join) as b1, neighbourhoods n
left outer join spatial_join b on b.area_id = n.area_id;
'''

pgquery(conn, query, msg="score_bike")


#Adding all the score together
query= '''
create view final_score as
select n.area_id,n.name,coalesce(s1.score ,0)+s2.z_score+s3.z_score+coalesce(s4.retail_score,0)+coalesce(s4.food_score,0)+coalesce(s4.health_score,0)+coalesce(s4.ed_score,0)+coalesce(s4.art_score,0) as "final z score"
from neighbourhoods n
left outer join score_bike s1 using (area_id)
left outer join score_dwelling s2 using (area_id)
left outer join score_pop s3 using(area_id)
left outer join score_service s4 using (area_id)


'''
pgquery(conn, query, msg="final_score")


# Task 4: DATA2901 Task for Advanced Class Only

In [ ]:
query='''Select * from final_score;'''

df=pd.read_sql_query(query,conn)

In [ ]:


df.to_csv("z_score_correct.csv")


data_Neighbourhood = pd.read_csv(open("z_score_correct.csv"))

data_Neighbourhood.head()



In [ ]:
suburbs = []
for i in range(len(data_Neighbourhood['name'])):
    t = data_Neighbourhood['name'][i].split(' - ')
    for sub in t:
        suburbs.append([data_Neighbourhood['area_id'][i],sub])

In [ ]:
def get_mentions(forum_text, suburbs):
    rem_list = ['North', 'South', 'East', 'West']
    mentions = []
    for entry in forum_text:
        [mentions.append(suburb) for suburb in suburbs if suburb[1] in entry.text and suburb[1] not in rem_list] 
    # above line takes away N,E,S,W loses some reference as these could be useful.
    return mentions

In [ ]:
base_url = "http://www.bicycles.net.au/forums/viewtopic.php?t=89780"
webpage_source = requests.get(base_url).text
page_content = BeautifulSoup(webpage_source, 'html5lib')
num_pages = int(page_content.find(class_="pagination").find('ul').find_all('li')[-2].text)
overall_mentions = []
# define suburbs again

for i in range(num_pages):
    if i == 0:
        first_page = False
        whole_text = page_content.find_all(class_ = 'content')
        first_page_forum_text = whole_text[1:]
        mentions = get_mentions(first_page_forum_text, suburbs)
        [overall_mentions.append(i) for i in mentions]
    else:
        try:
            webpage_source = requests.get(base_url + '&start=' + str(i*25)).text
            page_content = BeautifulSoup(webpage_source, 'html5lib')
            forum_text = page_content.find_all(class_ = 'content')
            mentions = get_mentions(forum_text, suburbs)
            [overall_mentions.append(i) for i in mentions] 
        except BaseException:
            pass

In [ ]:
#print(overall_mentions)
area_codes, names = zip(*overall_mentions)
values, counts = np.unique(area_codes, return_counts=True)
sub_count =  [[values[i], counts[i]] for i in range(len(values))]


In [ ]:
try:
    sub_count.remove([117031337, 28])
    sub_count.remove([125041492, 13])
except ValueError:
    pass
final_sub_count = sub_count


In [ ]:
data_Neighbourhood['op_count'] = np.zeros(len(data_Neighbourhood))

for j in final_sub_count:
        data_Neighbourhood['op_count'][data_Neighbourhood['area_id'] == j[0]] = j[1]


In [ ]:
train_data = data_Neighbourhood[data_Neighbourhood['op_count'] > 0]
train_x = train_data['final z score']
train_y = train_data['op_count']

plt.scatter(train_x,train_y )
plt.title('Mention Count vs Z Score (Training Data)')
plt.xlabel('Z Score')
plt.ylabel('Mention Count')
plt.savefig('initial_mcvzs_plot.png')

In [ ]:
# or use the 4/5 predictor variables used in the z-score
model = smf.ols(formula='train_y ~ train_x', data=train_data) 
reg = model.fit()
reg.summary()

In [ ]:
x = np.linspace(-15,25, 1000)
y_hat = 1.9104 + 0.0415 * x
plt.plot(x, y_hat)
plt.scatter(data_Neighbourhood['final z score'], data_Neighbourhood['op_count'])
plt.title('Mention Count vs Z Score (Reg Line and Unclassified Data)')
plt.xlabel('Z Score')
plt.ylabel('Mention Count')
plt.savefig('with_reg_mcvzs_plot.png')

In [ ]:
test_data = data_Neighbourhood[data_Neighbourhood['op_count'] == 0]
test_data['op_count'] = (1.9104 + 0.0415 * test_data['final z score']).round(0)
data_Neighbourhood['op_count'][data_Neighbourhood['op_count'] == 0] = test_data['op_count']

In [ ]:
x = np.linspace(-15,27, 1000)
y_hat = 1.9104 + 0.0415 * x
plt.plot(x, y_hat)
plt.scatter(data_Neighbourhood['final z score'], data_Neighbourhood['op_count'])
plt.title('Mention Count vs Z Score (Classified Data)')
plt.xlabel('Z Score')
plt.ylabel('Mention Count')
plt.savefig('classified_mcvzs_plot.png')

In [ ]:
ave = np.mean(data_Neighbourhood['op_count'])
std = np.std(data_Neighbourhood['op_count'])

data_Neighbourhood['extra_z_score'] = (data_Neighbourhood['op_count'] - ave)/std
data_Neighbourhood['final_z_score'] = data_Neighbourhood['final z score'] + data_Neighbourhood['extra_z_score']

In [ ]:
data_Neighbourhood.to_csv('z_score_final.csv')

# Upload own dataset and final z-score

In [ ]:
zscore = pd.read_csv("z_score_final.csv",index_col=False)
category = np.arange(-4,5,1)*np.std(zscore['final_z_score'])
point = np.arange(1,11,1)

In [ ]:
value = np.zeros(len(zscore['final_z_score']))

for i in range(len(value)):
    
    for j in range(len(category)):
        if zscore['final_z_score'][i] < category[j]:
            value[i] = point[j]
            break
        elif zscore['final_z_score'][i] > category[-1]:
            
            value[i] = point[-1]
            break
            


In [ ]:
zscore['op_count'] = pd.to_numeric(zscore['op_count'],downcast='integer')
zscore['point']=value
zscore['point'] = pd.to_numeric(zscore['point'],downcast='integer')
zscore.to_csv("z_score_new.csv")

dict = list(csv.DictReader(open("z_score_new.csv")))


In [ ]:
query='''
    Create table Z_Score(
        area_id int,
        name varchar(100),
        z_score float,
        op_count int,
        op_z_score float,
        final_z_score float,
        point int,
        foreign key (area_id) references StatisticalAreas(area_id)
    
    
    )

'''

pgquery(conn, query, msg="Z_Score")

In [ ]:
for i in dict:
    query ="Insert into Z_score(area_id,name,z_score,op_count,op_z_score,final_z_score,point) values(%(area_id)s,%(name)s,%(final z score)s,%(op_count)s,%(extra_z_score)s,%(final_z_score)s,%(point)s)"
    pgexec(conn,query,i,"Row copied")

# Correlation

In [ ]:
#Correlation analysis
    

#Create table for it
query='''
create table Corr(
area_id int,
name varchar(50),
z_score float,
income int,
rent int
);

INSERT INTO Corr(area_id)
SELECT area_id FROM Z_Score;
'''
pgquery(conn, query, msg="Corr")



query='''
UPDATE Corr
SET name = (SELECT z.name 
                     FROM Z_Score z WHERE z.area_id = Corr.area_id )
'''
pgquery(conn, query, msg="corr_table_update1")


query='''
UPDATE Corr
SET z_score = (SELECT z.final_z_score
                     FROM Z_Score z WHERE z.area_id = Corr.area_id )
'''
pgquery(conn, query, msg="corr_table_update2")


query='''
UPDATE Corr
SET income = (SELECT c.income
                     FROM CensusStats c WHERE c.area_id = Corr.area_id )
'''
pgquery(conn, query, msg="corr_table_update3")


query='''
UPDATE Corr
SET rent = (SELECT c.rent
                     FROM CensusStats c WHERE c.area_id = Corr.area_id )
'''
pgquery(conn, query, msg="corr_table_update4")


query='''
SELECT corr(z_score, income)
FROM Corr
'''
corr_income = pgquery(conn, query, msg="corr() study of score and income")
print(corr_income)


query='''
SELECT corr(z_score, rent)
FROM Corr
'''
corr_rent = pgquery(conn, query, msg="corr() study of score and rent")
print(corr_rent)

query='''
SELECT * FROM Corr'''
df = pd.read_sql_query(query,conn)
df.to_csv("corr.csv",sep=',')
corr_data = list(csv.DictReader(open("corr.csv")))


In [ ]:
#defin lists for ploting
score = []
rent = []
income = []

#Fill the lists
for i in corr_data:
    score.append(float(i['z_score']))
    rent.append(int(i['rent']))
    income.append(int(i['income']))

#Plot a scatter graph by rent and score
plt.scatter(score, rent, color='r',marker='X',s=0.5)
plt.title('Median Rent vs. Z score')
plt.xlabel('Z Score')
plt.ylabel('Median Rent')
plt.savefig("Rent_z_score.png", dpi =100)
plt.show()

In [ ]:
sns.regplot(x = score, y = rent)
plt.title('Median Rent vs. score')
plt.xlabel('Score')
plt.ylabel('Median Rent')
plt.savefig("Rent_z_score.png", dpi =100)
plt.show()

In [ ]:
#Plot a scatter graph by income and score
plt.scatter(score, income, color='r',marker='X',s=0.5)
plt.title('Median Income vs. Z Score')
plt.xlabel('Z Score')
plt.ylabel('Median Income')
plt.savefig("Income_z_score.png",dpi=100)
plt.show()

In [ ]:
sns.regplot(x = score, y = income)
plt.title("Median Income vs Score")
plt.xlabel('Score')
plt.ylabel('Median Income')
plt.savefig("Income_z_score.png",dpi=100)
plt.show()

# Mapping

In [ ]:
query='''
alter table Neighbourhoods
add op_count int;

update neighbourhoods
set op_count = z.op_count
from z_score z where Neighbourhoods.area_id=z.area_id;

'''


pgquery(conn,query,msg="adding colums")

query='''alter table Neighbourhoods
            add point int;
         update neighbourhoods
             set point= z.point
             from z_score z where z.area_id=neighbourhoods.area_id;'''

pgquery(conn,query,msg="adding colums")

In [ ]:
query='''select area_id,name,point,Polygon(geom) from Neighbourhoods'''


neighbour = pd.read_sql_query(query,conn)

In [ ]:
query="""SELECT POLYGON(geom) FROM Neighbourhoods"""
arealist = pgquery(conn, query, msg=".")

new_arealist=[]
for j in arealist:
    for i in range(len(j)):
        new_arealist.append(Polygon(ast.literal_eval(j[i])))

In [ ]:
neighbour['polygon'] = new_arealist
crs = {'init': 'epsg:4283'}



In [ ]:
query= '''select station_id,name ,geom_pts from Bikesharingpods;

'''

bikepod =pd.read_sql_query(query,conn)

query="""SELECT Point(geom_pts) FROM Bikesharingpods"""
pods = pgquery(conn, query, msg=".")

new_pods=[]
for j in pods:
    for i in range(len(j)):
        new_pods.append(Point(ast.literal_eval(j[i])))
        
bikepod['geom_pts'] =new_pods

In [ ]:
bike_gp = gpd.GeoDataFrame(bikepod,crs=crs,geometry=bikepod['geom_pts'])
gp = gpd.GeoDataFrame(neighbour,crs=crs,geometry=neighbour['polygon'])

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 10))

gp.plot(ax=ax,column='point',figsize=(20,20),legend=True)
plt.xlim(150.95,151.35)
plt.ylim(-34.0,-33.7)
plt.title("Cyclability Score of Sydney Suburbs")
plt.savefig("Cyclability.png")
plt.show()

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 10))

gp.plot(ax=ax,column='point',figsize=(20,20),legend=True)
bike_gp.plot(ax=ax,c='r')
plt.xlim(150.95,151.35)
plt.ylim(-34.0,-33.7)
plt.title("Cyclability Score of Sydney Suburbs")
plt.savefig("Cyclability.png")
plt.show()

In [ ]:
query = '''
select name, final_z_score from Z_score
order by final_z_score desc
limit 5;

'''
pgquery(conn, query, msg=".")


In [ ]:
query = '''
select name, final_z_score from Z_score
order by final_z_score asc
limit 5;

'''
pgquery(conn, query, msg=".")